|      |      | 
| ---- | ---- | 
| Journal:|Sustainability (ISSN 2071-1050)|
|Manuscript ID:| sustainability-1485080|
|Type:|Article|
|Title:|NEVs Supply Chain Coordination with Financial Constraint and Demand Uncertainty|
|manuscripts:|P.7 - figure 6. Data generation code|
|Code authors:|Yongjian Li$^*$ |
|Last modify:|2021-01-12|
|version:|Python 3.6.5, sympy >= 1.3|

In [52]:
from sympy import *
from sympy.stats import *
import numpy as np
import pandas as pd
# init_printing(use_unicode=True)
import sympy
sympy.__version__

'1.9'

In [53]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [54]:
# Initial parameter values
parameter = {p: 10, c: 4, s: 3, w: 6, theta: 0.8, b: 3, B: 100, r: 0.05,
             mu: 100, sigma: 100}

#### (see manuscript P(7) figure 6.)

\begin{align}
&\theta^*=\frac{s B(1+r)(p-c-w r)+w(1+r)(p-s)\left[F^{-1}(\alpha)(w-c)-B(1+r)\right]}{p\left[F^{-1}(\alpha)(w-c)(p-s)+B(1+r)(s-c-w r)\right]}\\
&b^*=\frac{(p-s)(w-c)w F^{-1}(\alpha)(1+r)+B s(1+r)(s-c-w r)}{F^{-1}(\alpha)(w-c)(p-s)+B(1+r)(s-c-w r)}
\end{align}



- Suppose X is normally distributed with $\mu=100$,$\sigma=100$ 
- CDF,exp1: $F(x) = \alpha$

In [55]:
X = Normal(x, mu, sigma).subs(parameter)
exp1 = Eq(cdf(X)(x), alpha)
exp1

Eq(erf(sqrt(2)*(x - 100)/200)/2 + 1/2, alpha)

- inverse exp1: $F^{-1}(\alpha) = x ,\quad \{0\leqslant \alpha \leqslant 1\}$

In [56]:
i_cdf = solve(exp1, x)[0]
i_cdf

100*sqrt(2)*erfinv(2*alpha - 1) + 100

In [57]:
# Convert a SymPy expression into a function that allows for fast numeric evaluation.
f = lambdify(alpha, i_cdf, modules=['numpy', 'sympy'])

x_0 = [i/1000 for i in range(1, 1000)]

dt = {
    "x_0": x_0,
}

df = pd.DataFrame(dt)
df['f(x)'] = df['x_0'].map(f)
df1 = df.copy()
df1['f(x)'] = df1['f(x)'].map(int)
df1 = df1.drop_duplicates(subset=['f(x)']).reset_index(drop=True).copy()
df1

,x_0,f(x)
0,0.001,-209
1,0.002,-187
2,0.003,-174
3,0.004,-165
4,0.005,-157
...,...,...
422,0.995,357
423,0.996,365
424,0.997,374
425,0.998,387


In [58]:
df1.query("`f(x)`==0")

,x_0,f(x)
113,0.157,0


In [59]:
# df1.iloc[::10].to_csv("inverse(CDF)",index=None,sep='\t')

\begin{align}
p \cdot \theta^*=\frac{s B(1+r)(p-c-w r)+w(1+r)(p-s)\left[F^{-1}(\alpha)(w-c)-B(1+r)\right]}{p\left[F^{-1}(\alpha)(w-c)(p-s)+B(1+r)(s-c-w r)\right]} \cdot p
\end{align}

In [60]:
fa = s*B*(1+r)*(p-c-w*r)+w*(1+r)*(p-s)*(x *(w-c)-B*(1+r))
fb = ((w-c)*(p-s)*x + B*(1+r)*(s-c-w*r)) 
expr =fa/(fb*p)
expr

(B*s*(r + 1)*(-c + p - r*w) + w*(p - s)*(r + 1)*(-B*(r + 1) + x*(-c + w)))/(p*(B*(r + 1)*(-c - r*w + s) + x*(-c + w)*(p - s)))

In [61]:
df

,x_0,f(x)
0,0.001,-209.023230616781
1,0.002,-187.816173909548
2,0.003,-174.778138544499
3,0.004,-165.206980790220
4,0.005,-157.582930354890
...,...,...
994,0.995,357.582930354890
995,0.996,365.206980790220
996,0.997,374.778138544499
997,0.998,387.816173909548


In [62]:
expr1=expr.subs(parameter)

f2 = lambdify(x, expr1, modules=['numpy', 'sympy'])

#df -> alpha
df_theta = df.copy()

df_theta['f(x)'] = df_theta['f(x)'].map(f2)
df_theta

,x_0,f(x)
0,0.001,0.694484580495644
1,0.002,0.701406454459450
2,0.003,0.706451754790763
3,0.004,0.710634107517639
4,0.005,0.714307971957940
...,...,...
994,0.995,0.589441735474539
995,0.996,0.590311652992051
996,0.997,0.591352294493647
997,0.998,0.592685103895660


In [63]:
df_theta['f(x)'].max()

95.2269238197602

In [64]:
thetal

,x_0,f(x)
0,0.001,6.94484580495644
1,0.002,7.01406454459450
2,0.003,7.06451754790763
3,0.004,7.10634107517639
4,0.005,7.14307971957940
...,...,...
163,0.164,24.9483101346814
164,0.165,25.9993942784531
165,0.166,27.1711586471638
166,0.167,28.4856861029508


In [65]:
thetal = df_theta.query("0<=`f(x)`<=3 and index <=209").reset_index(drop=True)
thetal['f(x)'] = thetal['f(x)'] *10

thetal.to_csv("thetap_l",index=None,sep='\t')

In [41]:
thetar = df_theta.query("0<=`f(x)`<=1 and index >=99").reset_index(drop=True)
thetar['f(x)'] = thetar['f(x)'] *10
thetar.iloc[::10].to_csv("thetap_r",index=None,sep='\t')

In [42]:
thetal

,x_0,f(x)
0,0.001,6.94484580495644
1,0.002,7.01406454459450
2,0.003,7.06451754790763
3,0.004,7.10634107517639
4,0.005,7.14307971957940
...,...,...
94,0.095,9.75705025798716
95,0.096,9.80771075184015
96,0.097,9.85947415355999
97,0.098,9.91238025610211



\begin{align}
b^*=\frac{(p-s)(w-c)w F^{-1}(\alpha)(1+r)+B s(1+r)(s-c-w r)}{F^{-1}(\alpha)(w-c)(p-s)+B(1+r)(s-c-w r)}
\end{align}

In [18]:
# 计算出CDF的反函数值后
fx = (p-s)*(w-c)*w *x *(1+r) +B*s*(1+r)*(s-c-w*r)
fy = ((w-c)*(p-s)*x + B*(1+r)*(s-c-w*r)) 
expr_b =fx/fy
expr_b

(B*s*(r + 1)*(-c - r*w + s) + w*x*(-c + w)*(p - s)*(r + 1))/(B*(r + 1)*(-c - r*w + s) + x*(-c + w)*(p - s))

In [19]:
df

,x_0,f(x)
0,0.001,-209.023230616781
1,0.002,-187.816173909548
2,0.003,-174.778138544499
3,0.004,-165.206980790220
4,0.005,-157.582930354890
...,...,...
994,0.995,357.582930354890
995,0.996,365.206980790220
996,0.997,374.778138544499
997,0.998,387.816173909548


In [20]:
expr2=expr_b.subs(parameter)

b2 = lambdify(x, expr2, modules=['numpy', 'sympy'])

#df -> alpha
df_b = df.copy()

df_b['f(x)'] = df_b['f(x)'].map(b2)
df_b

,x_0,f(x)
0,0.001,6.15292990413274
1,0.002,6.13714317403985
2,0.003,6.12563634872282
3,0.004,6.11609764952118
4,0.005,6.10771866044680
...,...,...
994,0.995,6.39250130505807
995,0.996,6.39051728264971
996,0.997,6.38814388975133
997,0.998,6.38510414900990


In [24]:
df_b_l = df_b.query("0<=`f(x)`<=6.4 and index <=189").reset_index(drop=True)
df_b_l

,x_0,f(x)
0,0.001,6.15292990413274
1,0.002,6.13714317403985
2,0.003,6.12563634872282
3,0.004,6.11609764952118
4,0.005,6.10771866044680
...,...,...
165,0.166,1.53991118573457
166,0.167,1.24010667827438
167,0.168,0.901397195838828
168,0.169,0.515674747441123


In [43]:
df_b_l.to_csv("b_l",index=None,sep='\t')
df_b_l.shape

(170, 2)

In [66]:
df_b_r = df_b.query("`f(x)`>=6.15 and index >=169").reset_index(drop=True)
df_b_r.to_csv("b_r",index=None,sep='\t')
df_b_r.shape

(816, 2)

In [67]:
df_b_r

,x_0,f(x)
0,0.184,147.789875504652
1,0.185,59.6879929204787
2,0.186,39.2436556071045
3,0.187,30.1434546167007
4,0.188,24.9960134413139
...,...,...
811,0.995,6.39250130505807
812,0.996,6.39051728264971
813,0.997,6.38814388975133
814,0.998,6.38510414900990


- some help fun() note here.

In [5]:
## note here
## Given a value, calculate the cumulative probability
value = 0
cdf(X)(value).evalf()
## Given a value, calculate the cumulative probability
value = 189.380
P(X < value).evalf(subs=parameter)